# 데이터 병합 및 전처리

In [2]:
import pandas as pd

import pandas as pd

# 인코딩을 명시적으로 지정해줘야 함
movies = pd.read_csv("movies.dat", sep="::", engine="python", encoding="latin1",
                     names=["movie_id", "title", "genres"])
ratings = pd.read_csv("ratings.dat", sep="::", engine="python", encoding="latin1",
                      names=["user_id", "movie_id", "rating", "timestamp"])
users = pd.read_csv("users.dat", sep="::", engine="python", encoding="latin1",
                    names=["user_id", "gender", "age", "occupation", "zip_code"])


# 연령 매핑
age_map = {
    1: "Under 18", 18: "18-24", 25: "25-34",
    35: "35-44", 45: "45-49", 50: "50-55", 56: "56+"
}
users["age_group"] = users["age"].map(age_map)

# 병합
df = ratings.merge(users, on="user_id").merge(movies, on="movie_id")

# 장르 분리
df_exploded = df.copy()
df_exploded["genres"] = df_exploded["genres"].str.split("|")
df_exploded = df_exploded.explode("genres")

# 연령대별 장르별 통계
summary = df_exploded.groupby(["age_group", "genres"]).agg(
    watch_count=("rating", "count"),
    avg_rating=("rating", "mean")
).reset_index()

# 저장
summary.to_csv("연령대별_장르_선호_및_평점.csv", index=False)
